# 基于Scikit-Learn API的 Classification模板

>20221110sym新建，从原来Jacs Cu体系的程序中借鉴，综合多种分类模型看特征重要性和全集表现

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import time
c_time = time.strftime("%Y%m%d_%H%M%S", time.localtime())
c_time_m = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [ ]:
# 参数
# ======== System Setup ========
Version = 'Classification-GBRT-sym-SeqFeat'
EPOCH = 16
CORE_NUM = 32       # 总运行轮数=EPOCH/CORE_NUM
# ======== Fit Data Input ========
S_N = 237
F_N = 54
INPUT_X = 'Features_'+str(S_N)+'_'+str(F_N)+'.csv'
INPUT_Y = 'Values_True_'+str(S_N)+'.csv'
INPUT_TITLE = 'Title_'+str(F_N)+'.csv'
INPUT_Xtest=None

BEGIN_INDEX = 0
END_INDEX_PLUS_ONE = None   # 设为None或者调成0和特征数量，即可使用全部特征
INPUT_SMILES = 'Smiles_'+str(S_N)+'.csv'

# ======== Other Fitting Settings ========
TRAIN_TEST_SPLIT = 0.85
SORT_SAMPLE = False
TEST_SPLIT_OOB = False
R2_HIGHER_LIMIT = 0.70
CONFIDENCE = 0.95
SAVE_RESULTS_OF_EVERY_ROUND = False
# ======== Data Output ========
SAVE_NAME = 'XGBoostClassification_'+c_time+'.png'
SUPTITLE = 'XGBoost on '+INPUT_X+' and '+INPUT_Y+'\nEPOCH:'+str(EPOCH)+'\n'

In [ ]:
X = np.loadtxt(INPUT_X, delimiter=',')[:, BEGIN_INDEX:END_INDEX_PLUS_ONE]
title = np.loadtxt(INPUT_TITLE, dtype='str', delimiter=',', comments='!')[BEGIN_INDEX:END_INDEX_PLUS_ONE, ]
y = np.loadtxt(INPUT_Y)
print('X:', X.shape, '   y:', y.shape)
headers=title
classbound=5
y=y>classbound
if INPUT_Xtest!=None:
    test = np.loadtxt(INPUT_Xtest, delimiter=',')[:, BEGIN_INDEX:END_INDEX_PLUS_ONE]
    # gbct.predict(test)用这个句式即可预测

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc
import pandas as pd
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import joblib
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression


In [ ]:
import os
from pathlib import Path
DIR = 'Classification'+Version+str(X.shape[1])+'_Fs_'+c_time
os.mkdir(DIR)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=10)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.15)
print('length of trainingset is'+str(len(X_train)))
print('    length of test is '+str(len(y_test)))
save_name = 'Xtrain_full_'+str(len(X_train))+'.csv'
save_name = Path('.', DIR, save_name)
np.savetxt(save_name, X_train, fmt='%s', delimiter=',')

In [ ]:
LOG_NAME = Version+c_time+'.txt'
LOG_NAME = Path('.', DIR, LOG_NAME)
f1 = open(LOG_NAME, 'w+')
f1.write('GBCT Regressor\n\n')
f1.write('Total Epoch: '+str(EPOCH)+'\n\n')
f1.write('Dataset: '+INPUT_X+', '+INPUT_Y+'\n')
f1.write('Data Shape: '+str(X.shape)+', '+str(y.shape)+'\n\n')
f1.write('y=y> '+str(classbound)+'\n\n')
f1.write('length of trainingset is'+str(len(X_train))+'    length of test is '+str(len(y_test)))

In [ ]:
#GBCT model
from sklearn.ensemble import GradientBoostingClassifier
gbct= GradientBoostingClassifier(n_estimators=500,learning_rate=0.0025,max_depth=5,verbose=0, validation_fraction=0.15, n_iter_no_change=50, tol=0.0001)
gbct.fit(X_train, y_train)
acc_unique = gbct.score(X_test, y_test)
print('Current accuracy:', acc_unique)
f1.write('GBCT current accuracy:\n'+str(acc_unique)+'\n')
f1.write('Now begin Recursive feature elimination'+'\n')
import joblib
save_name = 'Full_model.m'
save_name = Path('.', DIR, save_name)
joblib.dump(gbct,save_name)

In [ ]:
# Recursive feature elimination with cross-validation to select features from sklearn
min_features_to_select = 1  # Minimum number of features to consider

featnumset=10
count=0
featnumlist=[featnumset]
while featnumlist[-1]>=featnumset:
    count+=1
    f1.write(' Recursive feature elimination: '+str(count)+'begin\n')
    rfecv = RFECV(estimator=gbct,step=3,cv=cv,scoring='accuracy',min_features_to_select=min_features_to_select,n_jobs=CORE_NUM,importance_getter='feature_importances_')
    rfecv.fit(X, y)

    title_new=rfecv.get_feature_names_out(title)
    print(title_new)
    f1.write(' Recursive feature elimination title new: '+str(title_new)+'\n')
    print(f"Optimal number of features: {rfecv.n_features_}")
    featnumlist.append(rfecv.n_features_)
    data_out = []
    title_out = []
    for i in range(len(title)):
        t = title[i]
        if t in title_new:
            title_out.append(t)
            data_out.append(i)
    print(len(title_out))
    f1.write('This round is:'+str(count)+'\n'+'length of features is'+str(len(title_out))+'\n'+'feature list is ,'+str(title_out))
    X=X[:, data_out]
    title=title_out
    # drawpics
    n_scores = len(rfecv.cv_results_["mean_test_score"])
    fig, (ax1) = plt.subplots(figsize=(15, 15))
    plt.xlabel("Number of features selected", fontsize=25)
    plt.ylabel("Mean test accuracy", fontsize=25)
    plt.errorbar(
        range(min_features_to_select, n_scores + min_features_to_select),
        rfecv.cv_results_["mean_test_score"],
        yerr=rfecv.cv_results_["std_test_score"],
    )
    save_name = 'Recursive_GBCT_'+str(round(count,2))+'.csv'
    save_name = Path('.', DIR, save_name)
    np.savetxt(save_name, np.vstack((range(min_features_to_select, n_scores + min_features_to_select), rfecv.cv_results_["mean_test_score"], rfecv.cv_results_["std_test_score"]))
               , fmt='%s', delimiter=',')
    
    plt.title("Recursive Feature Elimination \nwith correlated features", fontsize=25)
    plt.xticks(fontname="Times New Roman", fontsize=25)
    plt.yticks(fontname="Times New Roman", fontsize=25)
    fig.tight_layout()
    plot_tree_name_2 = 'Recursive Feature Elimination process'+'_count_'+str(round(count,2))+'.png'
    plot_tree_name_2 = Path('.', DIR, plot_tree_name_2)
    plt.savefig(plot_tree_name_2, bbox_inches='tight',dpi=200)
    plt.show()

Because The shape of X changed by recursive algorithm, we have to train again

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
save_name = 'Xtrain_sel_'+str(len(X_train))+'.csv'
save_name = Path('.', DIR, save_name)
np.savetxt(save_name, X_train, fmt='%s', delimiter=',')
np.savetxt(save_name, X_train, fmt='%s', delimiter=',')
gbct.fit(X_train, y_train)
acc_unique = gbct.score(X_test, y_test)
print('Current accuracy:', acc_unique)
print(gbct.feature_importances_)
f1.write('\n\n   Classification-GBCT-RecursiveFeat_sym Done Normally at '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+'\n\n\n')
save_name = 'Sel_model.m'
save_name = Path('.', DIR, save_name)
joblib.dump(gbct,save_name)

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(gbct, X, y, n_repeats=10,
                                random_state=19)
print(result)
f1.write('GBCT 10 times permutation feature importance:\n'+str(result)+'\n')

perm_sorted_idx = result.importances_mean.argsort()
fig, (ax1) = plt.subplots(figsize=(15, 10))
ax1.boxplot(result.importances[perm_sorted_idx].T, vert=False,
          labels=np.array(title)[perm_sorted_idx])
plt.title("GradientBoostingClassifier model feature importance",fontname="Times New Roman", fontsize=25)
plt.xlabel('Feature importance', fontdict={"family": "Times New Roman", "size": 25})
plt.ylabel('Feature names', fontdict={"family": "Times New Roman", "size": 25})
plt.xticks(fontname="Times New Roman", fontsize=20)
plt.yticks(fontname="Times New Roman", fontsize=25)
fig.tight_layout()
plot_tree_name_2 = 'GradientBoostingClassifier Model Permutation Feature importance_'+'_acc_'+str(round(acc_unique,2))+'.png'
plot_tree_name_2 = Path('.', DIR, plot_tree_name_2)
plt.savefig(plot_tree_name_2, bbox_inches='tight',dpi=200)
plt.show()

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(gbct, X_test, y_test)  # doctest: +SKIP

modelname='GradientBoostingClassifier'
plt.title(modelname)
PLOT_NAME2 = 'confusion_matrix_'+modelname+'.png'
PLOT_NAME2 = Path('.', DIR, PLOT_NAME2)
plt.savefig(PLOT_NAME2, bbox_inches='tight',dpi=150)
plt.show()  # doctest: +SKIP

In [ ]:
data_out = []
title_out = []
for i in range(len(headers)):
    t = headers[i]
    if t in title:
        title_out.append(t)
        data_out.append(i)
print(len(title_out))
print(data_out)

f1.write('Pred_sel result is:'+str(predres2)+'\n')
f1.close()

In [ ]:
import shap
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.15, random_state=19)
explainer = shap.TreeExplainer(gbct)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X,feature_names=title)
shap_values
save_name = 'SHAP_Matrix_GBCT_'+c_time+'.csv'
save_name = Path('.', DIR, save_name)
np.savetxt(save_name, shap_values, fmt='%s', delimiter=',')
save_name = 'X_Matrix_GBCT_'+c_time+'.csv'
save_name = Path('.', DIR, save_name)
np.savetxt(save_name, X, fmt='%s', delimiter=',')
for i in range(len(title)):    
    plt.scatter(X[:,i], shap_values[:,i])
    plt.title(title[i])
    plt.show()

In [ ]:
# Run classifier with cross-validation and plot ROC curves

classifier = gbct

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

# plt.figure(figsize=(16, 16), dpi=250)
fig, ax = plt.subplots(figsize=(6, 6), dpi=250)
for i, (train, test) in enumerate(cv.split(X, y)):
    classifier.fit(X[train], y[train])
    viz = RocCurveDisplay.from_estimator(
        classifier,
        X[test],
        y[test],
        name="ROC fold {}".format(i),
        alpha=0.3,
        lw=1,
        ax=ax,
    )
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)


ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05]

)
plt.title("GradientBoostingClassifier",fontname="Times New Roman", fontsize=25)
plt.xlabel('False positive rate', fontdict={"family": "Times New Roman", "size": 25})
plt.ylabel('True positive rate', fontdict={"family": "Times New Roman", "size": 25})
plt.xticks(fontname="Times New Roman", fontsize=25)
plt.yticks(fontname="Times New Roman", fontsize=25)
ax.legend(loc="lower right")

score1=cross_validate(classifier, X, y, cv=cv, scoring=('accuracy','f1'))

print(score1)
PLOT_NAME6 = 'Receiver operating characteristic_'+'GradientBoostingClassifier Model_'+'_AUC_'+str(round(np.mean(mean_auc),3))+'_acc_'+str(round(np.mean(score1['test_accuracy']),3))+'_f1_'+str(round(np.mean(score1['test_f1']),3))+'.png';
PLOT_NAME6 = Path('.', DIR, PLOT_NAME6)
print(PLOT_NAME6)
plt.savefig(PLOT_NAME6, dpi=250, bbox_inches='tight')
plt.show()